# scRNA-Seq tutorial with the _scanpy_ package

This is a basic tutorial explaining a 10X scRNA-Seq data analysis pipeline using the scanpy package.
It includes short descriptions of AnnData objects and scanpy functionalities, including:

- data loading and subsetting
- data quality control
- normalisation and data transformation
- PCA
- clustering
- dimensionality reduction techniques
- scanpy plotting functions
- finding marker genes for each cluster
- simple trajectory analysis with PAGA

Loading required modules

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

import random
random.seed(123)

sc.settings.verbosity = 3

In [ ]:
#Sets figure size
from matplotlib import rcParams
rcParams['figure.figsize'] = (4, 4)

## Loading data

Loading a previously prepared .h5ad file (an hdf5 file)

The CellRanger pipeline outpus:
- sparse matrix format - a folder with: matrix.mtx, genes.tsv and barcodes.tsv. These can be imported with the function: sc.read_10x_mtx(path)
- hdf5 file format - filtered_feature_bc_matrix.h5, These can be imported with the function: sc.read_10x_h5(file)

In [ ]:
adata = sc.read('./LK_example2k.h5ad')
adata.var_names_make_unique()

Looking inside the object

In [ ]:
adata

We have 2000 cells (aka observations, rows of the matrix) and 27998 genes (aka variables, columns of the matrix)

## AnnData objects

**Attributes**

A detailed description of the AnnData object can be found here: https://icb-anndata.readthedocs-hosted.com/en/stable/index.html. Here is a brief summary:

.X - ndarray or sparray, a matrix containing counts

.obs - a pandas DataFrame with information about cells, cell IDs are stored in the DataFrame index

.var - a pandas DataFrame with information about features/genes, their IDs are stored in the DataFrame index

.uns - unstructured annotation, e.g. cluster colors

.obsm - multi-dimensional observations annotation for each cell - e.g. PCA or UMAP coordinates

**Methods**

Additionally AnnData objects include multiple methods:

adata.write() - saves the AnnData object to .h5ad format (using compression = 'lzf' saves a lot of space)

adata.concatenate() - concatenate two or more AnnData objects

adata.T() - transpose the object (swap genes and cells)

adata.copy() - make a copy of the object (important!)

**Subsetting**

Selecting cells/genes uses a standard notation [a,b], the variables can be: cell/gene names, numeric indexes or boolean values. For example here we select the first 100 cells and all genes.

In [ ]:
a = adata[0:100,:].copy()

Or selecting the 200:300 genes and all cells

In [ ]:
b = adata[:,200:300].copy()

**Adding additional annotation**

Let's imagine that we have some additional cell or gene annotations we would like to add to the AnnData object. It's stored in a .csv file called: anno.csv. We will load this data using pandas functionality and merge with the respective attribute of the AnnData object.

In [ ]:
anno = pd.read_csv('./anno.csv', index_col = 0) #index_col forces pandas to use the first column as index
anno

Note that the DataFrame index contains the cell names compatible with the AnnData object. Using the merge() mehod to join the .obs DataFrame with the annotation

In [ ]:
adata.obs = adata.obs.merge(anno, left_index = True, right_index = True)
adata

Analogously we can add gene annotation to the .var attribute

**Cocatenating AnnData objects**

If we want to join two datasets together we can use the built in AnnData method - concatenate. For simplicity we concatenate two the same objects - adata, and we get an ojbect with 4000 cells.

In [ ]:
adata.concatenate(adata)

## _scanpy_ functions

_scanpy_ functions are grouped accordingly:

sc.read - loading data

sc.pp - preprocessing functions (cell/gene filtering), normalisation, scaling etc

sc.tl - various tools like dimensionality reduction, finding nearest neighbors etc

sc.pl - plotting fuctions

**Important**: these function by default modify the AnnData **in place**, to avoid that behaviour set copy = True

## Preprocessing

### Quality control

scRNA-Seq experiments contain varying fractions of so called 'low-quality cells'. These can have various origins, including (but not limited to): dying or damaged cells, cell fragments or failed reactions. These cells usually have lower number of genes or UMIs detected and/or higher fraction of counts mapped to the mitochondrial genes (especially dying cells). To find these cells and filter them out we  calculate the number of counts/genes per cell and number of cells with >0 counts for each gene. The information is added to the .obs and .var attributes.

**Import note** - some cell types may naturally appear as low-quality, due to loow RNA content. An example of such population is HSCs. Thus it's best to choose thresholds with care.

In [ ]:
sc.pp.filter_cells(adata, min_genes = 0)
sc.pp.filter_cells(adata, min_counts = 0)
sc.pp.filter_genes(adata, min_cells = 0)

In [ ]:
adata

Plotting the distribution of values above

In [ ]:
n, bins, patches = plt.hist(np.log10(adata.obs.n_counts+1), 50, density=1, facecolor='green', alpha=0.75)
plt.title('No of counts per cell (log10)')
plt.show()

n, bins, patches = plt.hist(adata.obs['n_genes'], 50, range = [0,8000], density=1, facecolor='green', alpha=0.75)
plt.title('No of genes per cell')
plt.show()

To filter out low quality cells, we use the same functions filter_cells or filter_genes with specific thresholds.

In [ ]:
sc.pp.filter_cells(adata, min_genes = 2000)
sc.pp.filter_cells(adata, min_counts = 1000)

In [ ]:
adata

Identifying the mitochondrial genes (thir name starts with mt_)

In [ ]:
import re
mts = list(filter(lambda x: re.search('^mt-', x), adata.var.index))
print(mts)

Counting the UMIs mapped to mitochondrial genes and plotting them

In [ ]:
mt = adata[:,mts]
adata.obs['mt_count'] = mt.X.sum(axis = 1).A1
adata.obs['mt_frac'] = adata.obs['mt_count'] / adata.obs['n_counts']

In [ ]:
n, bins, patches = plt.hist(adata.obs['mt_frac'], 50, range=[0, 0.2], density=1, facecolor='green', alpha=0.75)
plt.title('Fraction of counts in mitochondrial genes')
plt.show()

Usual threshold here is 10%, here all cells pas the threshold

In [ ]:
print('Cells with mitochondrial counts> 10%:' + str(sum(adata.obs['mt_frac'] > 0.1)))
adata = adata[~(adata.obs['mt_frac'] > 0.1),:]
print(adata)

### Normalisation etc

First we remove genes without any expression across the whole dataset

In [ ]:
sc.pp.filter_genes(adata, min_counts=1)

For preprocessing the typical order is: normalise data (here a simple normalisation per 10000 counts), log-transform, select variable features/genes and scale

Let's perform the operations and see how the values change.
Note: todense() is required to covert the sparse matrix input

In [ ]:
adata.X[100:105, 102:107].todense()

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after = 10000)
adata.X[100:105, 102:107].todense()

In [ ]:
sc.pp.log1p(adata)
adata.X[100:105, 102:107].todense()

A large fraction of genes is not informative for our analysis and adds unnecessary noise to the data. Hence we will select top 5000 gene with the highest dispersion

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='seurat', n_top_genes = 5000)
print("No of variable genes detected: " + str(sum(adata.var.highly_variable)))
sc.pl.highly_variable_genes(adata)

The variable genes are labelled as highly_variable in the .var slot, and by default are used for subsequent computations

In [ ]:
adata

Before scaling the data, we will save the log-normalised data

In [ ]:
adata.write('LK_example2k_nlog.h5ad', compression = 'lzf')

Scaling centers data around the mean expression values. This essentially gives equal weight to each gene for other computations

In [ ]:
sc.pp.scale(adata)
adata.X[100:105, 102:107] #Note the lack of todense(), scaling converts the .X to a normal matrix

Now we can load back the log-normalised data into the .raw slot of the AnnData object. These counts are by default not used for the downstream computation but are accessed by default to plot gene expression or for differential expression.

In [ ]:
adata.raw = sc.read('LK_example2k_nlog.h5ad')

## Data transformation

#### Principal component analysis

The first step is to reduce the 5000 dimensions (variable genes) to a more managable number. Principal component analysis find directions in the high-dimensional space starting with those with the highest variance. Thus selecting the top principal components allows us to represent a large fraction of the variability in the data. Here we compute the first 50 components.

In [ ]:
sc.tl.pca(adata, n_comps=50, svd_solver = 'arpack')

We can plot the first few components to get a sense of what is going on in the data

In [ ]:
sc.pl.pca(adata, color = ['Procr', 'Klf1', 'Elane', 'Dntt'], components = ['1,2', '2,3'])

How many PCs do we actually need? We can look at the variance explained by each component

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 50, log=False)

Loading indicate how much each gene contribute to each PC

In [ ]:
sc.pl.pca_loadings(adata)

We will use here the first 20 principal components

#### Finding nearest neighbors

Many of the dimensionality reduction techniques and the clustering algorithm rely on a nearest neighbor graph. We use the sc.pp.neighbors function and need to specify two key parameters: **number of neighbors for each cell** and the number of PCs to be used.

In [ ]:
sc.pp.neighbors(adata, n_neighbors = 7, n_pcs = 20)

#### Clustering

To cluster the data we will use the leiden algorithm, which is a community-detection algorithm trying to maximise the modularity. The resolution parameter tweaks how fine cluster we want to detect.

In [ ]:
sc.tl.leiden(adata, resolution=0.5)

### Dimensionality reduction techniques (other than the PCA)

#### Diffusion maps

Diffusion maps model the differentiation process as a diffusion process, thus they are particularly suitable for capturing trajectories in the data. Just like with PCA, multiple components, each with a decreasing 'significance', are computed. However, if multiple trajectories are present in the data, they will be distributed across multiple components and thus difficult to see in 2D.

In [ ]:
sc.tl.diffmap(adata)

In [ ]:
sc.pl.diffmap(adata, color = ['Procr', 'Klf1', 'Elane', 'Dntt'], components = ['1,2', '2,3'])

#### tSNE

tSNE is machine learning technique, which tries to find the best 2d (or 3d) representation of the data by optimising the nearby point distribution. However, the method largely neglects global structure of the data in the process. This means that it picks the clusters effectively, but their relative location may not represent progressions/relation in the higher dimensional space.
Note: the method is non-linear, hence distances between points do not represent distances in the gene expression space

Source: https://lvdmaaten.github.io/tsne/

In [ ]:
sc.tl.tsne(adata)

In [ ]:
sc.pl.tsne(adata, color = ['Procr', 'Klf1', 'Elane', 'Dntt', 'leiden'])

#### UMAP

UMAP is build on similar principles to the tSNE algorithm but with the aim to preserve the global topology of the data when finding a low-dimensional representation. It uses the nearst-neighbor graph to "map" the high-dimensional manifold and a machine learning approach to find the best 2D and 3D embedding.

Note: the method is non-linear, hence distances between points do not represent distances in the gene expression space
Source: https://umap-learn.readthedocs.io/en/latest/how_umap_works.html

In [ ]:
sc.tl.umap(adata)

In [ ]:
ax = sc.pl.umap(adata, color=['leiden', 'n_genes', 'n_counts'], legend_loc = 'on data', save = '_info.pdf', alpha = 0.5)
ax = sc.pl.umap(adata, color=['Procr', 'Klf1', 'Elane', 'Ms4a2', 'Pf4', 'Irf8', 'Dntt'], legend_loc = 'on data', save = '_markers.pdf', alpha = 0.5)

#### Force-directed graph (SPRING)

The method uses nearest-neighbor graphs which are laid out in 2D using repulsion and attraction forces. Briefly, all the nodes try to escape from each other while the edges between them keep them together. Method performs well even with complex landscapes, but can be sensitive to spurious edges, which don't allow the graph to 'open up'.

Source: https://kleintools.hms.harvard.edu/tools/spring.html

In [ ]:
sc.tl.draw_graph(adata)

In [ ]:
ax = sc.pl.draw_graph(adata, color=['leiden', 'n_genes', 'n_counts'], legend_loc = 'on data', save = '_info.pdf', alpha = 0.5)
ax = sc.pl.draw_graph(adata, color=['Procr', 'Klf1', 'Elane', 'Ms4a2', 'Pf4', 'Irf8', 'Dntt'], legend_loc = 'on data', save = '_markers.pdf', alpha = 0.5)

## Removing confounders

There are many ways of dealing with confounder variables in the scRNA-Seq data. Above we can see that the orientation of early erythroid and megakaryocytic trajectories look a bit funny and on top of that the erythroid trajectory is kind of split into two parts.

One of the main confounders in scRNA-Seq data is cell cycle. Here we used a fairly naive but effective approach to minimise its impact on our representations: we remove 368 genes related to the cell cycle process.

Making a copy of the AnnData object and removing the cell cycle genes from the highly variable list (which means they will not be used for any downstream computation

In [ ]:
adata_noCC = adata.copy()

In [ ]:
adata_noCC.var.loc[adata.var.isCC & adata.var.highly_variable, 'highly_variable'] = False

Recomputing the PCA and finding the nearest neighbors again

In [ ]:
sc.tl.pca(adata_noCC, n_comps=50, svd_solver = 'arpack')

In [ ]:
sc.pp.neighbors(adata_noCC, n_neighbors = 7, n_pcs = 20)

Clustering (again)

In [ ]:
sc.tl.leiden(adata_noCC, resolution=0.75)

UMAP (again)

In [ ]:
sc.tl.umap(adata_noCC)

In [ ]:
ax = sc.pl.umap(adata_noCC, color=['leiden', 'n_genes', 'n_counts'], legend_loc = 'on data', save = '_info.pdf', alpha = 0.5)
ax = sc.pl.umap(adata_noCC, color=['Procr', 'Klf1', 'Elane', 'Ms4a2', 'Pf4', 'Irf8', 'Dntt'], legend_loc = 'on data', save = '_markers.pdf', alpha = 0.5)

#### Force-directed graph

In [ ]:
sc.tl.draw_graph(adata_noCC)

In [ ]:
ax = sc.pl.draw_graph(adata_noCC, color=['leiden', 'n_genes', 'n_counts'], legend_loc = 'on data', save = '_info.pdf', alpha = 0.5)
ax = sc.pl.draw_graph(adata_noCC, color=['Procr', 'Klf1', 'Elane', 'Ms4a2', 'Pf4', 'Irf8', 'Dntt'], legend_loc = 'on data', save = '_markers.pdf', alpha = 0.5)

After removing the cell cycle genes both visualisations appear clearer.

## Plotting gene expression

In addition to color-coded scatter plots above we can summarise gene expression in various other ways.

#### Heatmaps

In [ ]:
sc.pl.heatmap(adata_noCC, var_names = ['Procr', 'Klf1', 'Elane', 'Ms4a2', 'Pf4', 'Irf8', 'Dntt'], 
              groupby = 'leiden', dendrogram = True)

#### Dotplots

In [ ]:
sc.pl.dotplot(adata_noCC, var_names = ['Procr', 'Klf1', 'Elane', 'Ms4a2', 'Pf4', 'Irf8', 'Dntt'], 
              groupby = 'leiden')

#### Violins

In [ ]:
sc.pl.violin(adata_noCC, keys = ['Procr', 'Klf1', 'Elane', 'Ms4a2', 'Pf4', 'Irf8', 'Dntt'], 
              groupby = 'leiden')

## Simple differential expression - finding markers

Scanpy implement a simple way for performing differential expression using the t-test. The default setting runs a comparison for each cluster against all other clusters combined, hence is particularly useful for detecting marker genes.

In [ ]:
sc.tl.rank_genes_groups(adata_noCC, groupby = 'leiden', reference = 'rest', n_genes = 100)

Plotting results

In [ ]:
sc.pl.rank_genes_groups(adata_noCC)

We can also specify the comparisons explicitly, for instance here we want narrow down the comparisons to clusters: 0,1,2 and compare them against the reference of cluster 0.

In [ ]:
sc.tl.rank_genes_groups(adata_noCC, groupby = 'leiden', groups = ['0', '1', '2'], reference = '0', n_genes = 100)

In [ ]:
sc.pl.rank_genes_groups(adata_noCC)

## Trajectory identification - PAGA

Trajectory inference in scRNA-Seq is fairly complex and dozens of various methods have been published on the subject. PAGA is broadly applicable and easy to use thanks to the implementation in the scanpy package. PAGA analysed the nearest neighbour graphs and provides a map of clusters based on their connectivity in the high-dimensional space.

source: https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1663-x

In [ ]:
sc.tl.paga(adata_noCC, groups = 'leiden')

In [ ]:
sc.pl.paga(adata_noCC)

In [ ]:
sc.pl.paga_compare(adata_noCC, basis = 'umap')

Essentially the edges connect clusters which are related to each other, thus allowing predictions about cell flow through the graph.

## Writing data

In [ ]:
adata_noCC.write('LK_example2k_processed.h5ad', compression = 'lzf')